In [163]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime
import time
import sys
import json
import psycopg2
import redis

In [41]:
reload(sys)
sys.setdefaultencoding('utf8') 

In [42]:
table1 = pd.read_excel("TAG_LOG_DOWNLOAD.xlsx")
table1.head()

,ID,UTID
0,1,TG0122
1,1,TG0124
2,1,TG0127
3,1,TG0128
4,1,TG0129


In [43]:
table1['TAG_VALUE'] = ""
table1.head()

,ID,UTID,TAG_VALUE
0,1,TG0122,
1,1,TG0124,
2,1,TG0127,
3,1,TG0128,
4,1,TG0129,


In [44]:
table2 = pd.read_excel("TAG_Value_DOWNLOAD.xlsx")
table2.head()

,ID,UTID,TAG_VALUE
0,1,SEG001,1
1,2,SEG001,2
2,3,SEG001,3
3,4,SEG001,4
4,5,SEG001,5


In [45]:
table_merge = pd.concat([table1.loc[:,:], table2.loc[:,:]], axis = 0)
table_merge = table_merge.reset_index(drop=True)

In [235]:
import redis
r = redis.Redis(host='localhost', port=6379, db=1)
unique_tagValue = list(np.unique(table2['TAG_VALUE']))
for value in unique_tagValue:
    r.set(value, offer_db_query(value, cur)['offer_list'])
r.get("2")

"[u'5_DreamEcard.png', u'6_CardAddFriend.png', u'14_Parking.png', u'7_AndroidPay.png', u'1_WomanEcard.png', u'8_FX_NBOpen.png', u'4_InvitMovie.png', u'3_NB_FXTrade.png', u'16_FX_Insurance.png', u'17_FundPopular.png']"

In [193]:
import psycopg2


# connect PostgreSQL
conn = psycopg2.connect(database = "MJ",
                        host = "localhost",
                        user = "llc",
                        password = "")
cur = conn.cursor()

In [166]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [70]:
def intent_db_query(UTID, sql_db):
    SQL = "SELECT tag_info FROM intent_db WHERE tag_info->>'TAG_ID' = '{}';".format(UTID)
    sql_db.execute(SQL)
    records = sql_db.fetchall()[0][0]
    return records

In [194]:
def offer_db_query(TAG_VALUE, sql_db):
    SQL = "SELECT tag_info FROM offer_db WHERE tag_info->>'Tag_value' = '{}';".format(TAG_VALUE)
    sql_db.execute(SQL)
    records = sql_db.fetchall()[0][0]
    return records

In [71]:
print intent_db_query("TG0122",cur)[u'Chinese_Desc']

In [73]:
import numpy as np
unique_ID = np.unique(table_merge['ID'])
len(unique_ID)

1500

In [86]:
unique_ID_table

,ID,UTID,TAG_VALUE
8939,1500,SEG001,7
10439,1500,SEG002,3


In [236]:
#tagging_Json = []

for i in unique_ID:
    temp_list = []
    temp_detail = {}    
    unique_ID_table = table_merge[table_merge.ID==i]
    temp_detail['VID'] = i
    temp_detail['realtimeTag'] = []
    #for utid in unique_ID_table['UTID']:
    #   temp_list.append(tag_db_query(utid,cur))
    for j in range(len(unique_ID_table)):
        query_update = {}
        query_temp = tag_db_query(unique_ID_table['UTID'].values[j],cur)

        query_update['scenario'] = query_temp['scenario']
        query_update['recommendWeight'] = query_temp['recommedWeight']
        query_update['dataSource'] = query_temp['dataSource']
        query_update['securityFilter'] = query_temp['securityFilter']
        query_update['tagValue'] = unique_ID_table['TAG_VALUE'].values[j]
        query_update['tagId'] = query_temp['TAG_ID']
        query_update['tagName'] = query_temp['Chinese_Desc']
        query_update['tagTime'] = time.time()
        query_update['expireTime'] = time.time() + float(query_temp['expireAfter'])
        query_update['isValid'] = query_temp['isActive']
        temp_list.append(query_update)
    temp_detail['batchTag'] = temp_list
    r.set(i, json.dumps(temp_detail,ensure_ascii=False))
    
#    tagging_Json.append(temp_detail)
#tagging_Json

In [237]:
temp_detail

{'VID': 1500,
 'batchTag': [{'dataSource': u'EBM',
   'expireTime': 1511427383.629948,
   'isValid': u'Y',
   'recommendWeight': u'1',
   'scenario': {u'DIIS1': u'S', u'DIIS2': u'SEG001', u'DIIS3': u'01'},
   'securityFilter': u'admin',
   'tagId': u'SEG001',
   'tagName': u'\u5ba2\u7fa41',
   'tagTime': 1511427353.629947,
   'tagValue': 7},
  {'dataSource': u'EBM',
   'expireTime': 1511427383.633504,
   'isValid': u'Y',
   'recommendWeight': u'1',
   'scenario': {u'DIIS1': u'S', u'DIIS2': u'SEG002', u'DIIS3': u'02'},
   'securityFilter': u'admin',
   'tagId': u'SEG002',
   'tagName': u'\u5ba2\u7fa42',
   'tagTime': 1511427353.633503,
   'tagValue': 3}],
 'realtimeTag': []}

In [ ]:
for i in intent_Json:
    SQL = "INSERT INTO tagging_db(tag_info) VALUES('{}' );".format(json.dumps(i))
    cur.execute(SQL)
    conn.commit()

In [173]:
r.get(1)

'{"batchTag": [{"tagTime": 1511251102.208681, "scenario": {"DIIS2": "SEG001", "DIIS3": "01", "DIIS1": "S"}, "isValid": "Y", "tagId": "SEG001", "securityFilter": "admin", "expireTime": 1511251132.208681, "tagValue": 7, "dataSource": "EBM", "tagName": "\xe5\xae\xa2\xe7\xbe\xa41", "recommendWeight": "1"}, {"tagTime": 1511251102.211655, "scenario": {"DIIS2": "SEG002", "DIIS3": "02", "DIIS1": "S"}, "isValid": "Y", "tagId": "SEG002", "securityFilter": "admin", "expireTime": 1511251132.211656, "tagValue": 3, "dataSource": "EBM", "tagName": "\xe5\xae\xa2\xe7\xbe\xa42", "recommendWeight": "1"}], "realtimeTag": [], "VID": 1500}'

In [215]:
json.loads(r.get(1))

{u'VID': 1,
 u'batchTag': [{u'dataSource': u'WB',
   u'expireTime': 1511251094.464498,
   u'isValid': u'Y',
   u'recommendWeight': u'1',
   u'scenario': {u'DIIS1': u'H', u'DIIS2': u'SHP', u'DIIS3': u'SHP04'},
   u'securityFilter': u'admin',
   u'tagId': u'TG0122',
   u'tagName': u'\u751f\u6d3b/\u751f\u6d3b\u767e\u8ca8/\u5b98\u7db2',
   u'tagTime': 1511251066.464497,
   u'tagValue': u''},
  {u'dataSource': u'WB',
   u'expireTime': 1511251094.469115,
   u'isValid': u'Y',
   u'recommendWeight': u'1',
   u'scenario': {u'DIIS1': u'H', u'DIIS2': u'SHP', u'DIIS3': u'SHP06'},
   u'securityFilter': u'admin',
   u'tagId': u'TG0124',
   u'tagName': u'\u5916\u532f/\u7db2\u9280',
   u'tagTime': 1511251066.469113,
   u'tagValue': u''},
  {u'dataSource': u'CC',
   u'expireTime': 1511251094.474415,
   u'isValid': u'Y',
   u'recommendWeight': u'1',
   u'scenario': {u'DIIS1': u'H', u'DIIS2': u'SHP', u'DIIS3': u'SHP02'},
   u'securityFilter': u'admin',
   u'tagId': u'TG0127',
   u'tagName': u'BEH\u5047\u

In [183]:
a.values()[0][0]['tagId']

u'SEG001'

In [214]:
def getTagValue(ID, tagID):
    for i in range(len(table2)):
        if table2['ID'][i] == ID and table2['UTID'][i] == tagID:
            return table2['TAG_VALUE'][i]
        
def getOffer(TAG_VALUE):
    return offer_db_query(TAG_VALUE, cur)['offer_list']

input1 = int(raw_input("input VID:"))
#tag_info = json.loads(r.get(input1))
input2 = raw_input("input tagId:")
getTagValue(input1, input2)

input VID:1
input tagId:'SEG001'


In [217]:
def getTagValue(ID, tagID):
    for i in range(len(table2)):
        if table2['ID'][i] == ID and table2['UTID'][i] == tagID:
            return table2['TAG_VALUE'][i]
        
def getOffer(TAG_VALUE):
    return offer_db_query(TAG_VALUE, cur)['offer_list']

getOffer(getTagValue(1,'SEG001'))

[u'1_WomanEcard.png',
 u'5_DreamEcard.png',
 u'6_CardAddFriend.png',
 u'12_WeddingLN.png',
 u'4_InvitMovie.png',
 u'7_AndroidPay.png',
 u'8_FX_NBOpen.png',
 u'3_NB_FXTrade.png',
 u'17_FundPopular.png',
 u'10_Ecard.png']

In [ ]:
"""
tagging_Json = []

for i in unique_ID:
    temp_list = []
    temp_detail = {}    
    unique_ID_table = table_merge[table_merge.ID==i]
    for utid in unique_ID_table['UTID']:
        temp_detail['VID'] = i
        temp_list.append(tag_db_query(utid,cur))
    temp_detail['batchTag'] = temp_list
    
    tagging_Json.append(temp_detail)
tagging_Json
"""
"""
tagging_Json = []

for i in range(table_merge.shape[0]):    
    temp_detail = {}
    temp_detail['VID'] = table_merge['ID'].values[i]
                         #str(table_merge.loc[i,:]['ID'])
    query_UTID = table_merge['UTID'].values[i]
                 #str(table_merge.loc[i,:]['UTID'])
    temp_detail['batchTag'] = tag_db_query(query_UTID,cur)
    
    tagging_Json.append(temp_detail)
tagging_Json
"""


In [170]:
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0)

for i in range(3):
    for j in range(len(list(table2))):
        r.hset(table2['ID'][i], list(table2)[j], table2[list(table2)[j]][i])

r.hgetall(3)

{'ID': '3', 'TAG_VALUE': '3', 'UTID': 'SEG001'}

In [135]:
"""
query_update = {}
query_temp = tag_db_query(unique_ID_table['UTID'].values[1],cur)

query_update['scenario'] = query_temp['scenario']
query_update['recommendWeight'] = query_temp['recommedWeight']
query_update['dataSource'] = query_temp['dataSource']
query_update['securityFilter'] = query_temp['securityFilter']
query_update['tagValue'] = unique_ID_table['TAG_VALUE'].values[1]
query_update['tagId'] = query_temp['TAG_ID']
query_update['tagName'] = query_temp['Chinese_Desc']
query_update['tagTime'] = time.time()
query_update['expireTime'] = time.time() + float(query_temp['expireAfter'])
query_update['isValid'] = query_temp['isActive']

query_update
"""

{'UTID': u'SEG002',
 'dataSource': u'EBM',
 'expireTime': 1511246809.328338,
 'isValid': u'Y',
 'recommendWeight': u'1',
 'scenario': {u'DIIS1': u'S', u'DIIS2': u'SEG002', u'DIIS3': u'02'},
 'securityFilter': u'admin',
 'tagId': u'SEG002',
 'tagName': u'\u5ba2\u7fa42',
 'tagTime': 1511246779.328193,
 'tagValue': 3}